In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
#import random
#import gurobipy as gp
#from gurobipy import GRB


In [5]:
'''
df_exR = pd.read_csv("exR_file.csv")
df_exR
print(df_exR)
ticker = list(df_exR)[1:] #the first is the time stamp
print(ticker)
'''

    formatted_date    XSP.TO    XIU.TO    XIN.TO    XBB.TO    XGB.TO  \
0       2006-04-01       NaN       NaN       NaN       NaN       NaN   
1       2006-05-01 -0.037262 -0.035934 -0.059806 -0.000070       NaN   
2       2006-06-01  0.002846 -0.014420  0.007787 -0.020828       NaN   
3       2006-07-01 -0.002867  0.018924 -0.001692  0.028914       NaN   
4       2006-08-01  0.017057  0.018122  0.024081  0.014012       NaN   
..             ...       ...       ...       ...       ...       ...   
178     2021-02-01  0.027248  0.036481  0.027873 -0.027362 -0.031432   
179     2021-03-01  0.044082  0.052970  0.050659 -0.012869 -0.014480   
180     2021-04-01  0.050627  0.023047  0.011021  0.000274  0.001319   
181     2021-05-01  0.004213  0.030843  0.022821  0.005995  0.005625   
182     2021-06-01  0.021196  0.033390  0.001299  0.010679  0.011981   

       XRB.TO    XRE.TO  
0         NaN       NaN  
1   -0.013703 -0.002474  
2   -0.021193 -0.009137  
3    0.040237  0.054973  
4    

In [2]:

def dataSlicing(df1,df2,df3,df4):

    
    #need to transfer to main code
    '''
    df1= df_exR.iloc[:,1:2]
    df2 = df_exR.iloc[:,2:3]
    df3 = df_exR.iloc[:,3:6]
    df4 = df_exR.iloc[:,6:8]
    '''
    asset_1_num = df1.shape[1]
    asset_2_num = df2.shape[1]
    asset_3_num = df3.shape[1]
    asset_4_num = df4.shape[1]
    
    asset_1_index = list(range(0 , asset_1_num))
    asset_2_index = list(range(asset_1_num , asset_1_num+asset_2_num))
    asset_3_index = list(range(asset_1_num+asset_2_num , asset_1_num+asset_2_num+asset_3_num))
    asset_4_index = list(range(asset_1_num+asset_2_num+asset_3_num ,asset_1_num+asset_2_num+asset_3_num+asset_4_num))
    
    '''
    asset_1_index = [0]
    asset_2_index = list(range(1,2))
    asset_3_index = list(range(2,5))
    asset_4_index = list(range(5,7))
    '''
    
    total_df = pd.concat([df1,df2,df3,df4], axis=1)# merge the 4 asset class
    ticker = list(total_df)        #[1:]#the first is the time stamp
    #print(total_df.shape[1]) # get the amount of columns of the dataframe
    
    return total_df,ticker

total_df = dataSlicing(df1,df2,df3,df4)

number of asset is  7


In [12]:
def formDf(x,ticker):
    x = pd.DataFrame(x).T
    x.columns = ticker
    return x

In [ ]:
# Equal Weight strategy for Block

def EW(df1,df2,df3,df4):
    
    total_df,ticker = dataSlicing(df1,df2,df3,df4)
    Q = total_df.cov()
    numAsset = total_df.shape[1]
    
    weight_eq = pd.DataFrame(np.repeat(1/numAsset, numAsset))
    x_ew = formDf(weight_eq,ticker) # result for equal weight
    return x_ew

EW(df1,df2,df3,df4)

In [15]:
# Mean Variance optimization Block
def meanVarianceOpt(numAsset, Q):
    def objective(weight):#weight
        weight = np.array(weight)
        return weight.dot(Q).dot(weight.T)

    #constraints
    cons = ({'type':"eq",'fun':lambda x: np.sum(x)-1},
           )

    #long only
    bounds = tuple((0,1) for x in range(numAsset))

    #initialize the weights with an even split, equal weights
    guess = pd.DataFrame(np.repeat(1/numAsset, numAsset))
    optimized_result = minimize(objective, guess, method = "SLSQP", bounds  = bounds, constraints= cons)

    return optimized_result

def MVO(df1,df2,df3,df4):
    total_df, ticker = dataSlicing(df1,df2,df3,df4)
    Q = total_df.cov()
    numAsset = total_df.shape[1]

    x_mvo = meanVarianceOpt(numAsset,Q).x
    x_mvo = formDf(x_mvo,ticker)
    return x_mvo

weight_mvo = MVO(df1,df2,df3,df4) #temperarily set for current data set


NameError: name 'df1' is not defined

In [ ]:
# Mean Variance optimization - All weather Block

def meanVarianceOpt_allWeather(numAsset,Q, asset_1_index, asset_2_index, asset_3_index, asset_4_index):
    def objective(weight):#weight
        weight = np.array(weight)
        return weight.dot(Q).dot(weight.T)

    #constraints
    cons = ({'type':"eq",'fun':lambda x: np.sum(x)-1},
            {"type":"ineq",'fun': lambda x: np.sum(np.array(x)[asset_1_index])-0.15 },
            {"type":"ineq",'fun': lambda x: np.sum(np.array(x)[asset_2_index])-0.15 },
            {"type":"ineq",'fun': lambda x: np.sum(np.array(x)[asset_3_index])-0.15 },
            {"type":"ineq",'fun': lambda x: np.sum(np.array(x)[asset_4_index])-0.15 },
            {"type":"ineq",'fun': lambda x: 0.3-np.sum(np.array(x)[asset_1_index]) },
            {"type":"ineq",'fun': lambda x: 0.3-np.sum(np.array(x)[asset_2_index]) },
            {"type":"ineq",'fun': lambda x: 0.3-np.sum(np.array(x)[asset_3_index]) },
            {"type":"ineq",'fun': lambda x: 0.3-np.sum(np.array(x)[asset_4_index]) },
           )

    #long only
    bounds = tuple((0,0.3) for x in range(numAsset))

    #initialize the weights with an even split, equal weights
    guess = pd.DataFrame(np.repeat(1/numAsset, numAsset))
    optimized_result = minimize(objective, guess, method = "SLSQP", bounds  = bounds, constraints= cons)

    return optimized_result

def MVO_AW(df1,df2,df3,df4):
    
    asset_1_num = df1.shape[1]
    asset_2_num = df2.shape[1]
    asset_3_num = df3.shape[1]
    asset_4_num = df4.shape[1]
    
    asset_1_index = list(range(0 , asset_1_num))
    asset_2_index = list(range(asset_1_num , asset_1_num+asset_2_num))
    asset_3_index = list(range(asset_1_num+asset_2_num , asset_1_num+asset_2_num+asset_3_num))
    asset_4_index = list(range(asset_1_num+asset_2_num+asset_3_num ,asset_1_num+asset_2_num+asset_3_num+asset_4_num))
    
    total_df, ticker = dataSlicing(df1,df2,df3,df4)
    Q = total_df.cov()
    numAsset = total_df.shape[1]
    
    x_mvo_allWeather = meanVarianceOpt_allWeather(numAsset,Q, asset_1_index,asset_2_index,asset_3_index,asset_4_index).x
    x_mvo_allWeather = formDf(x_mvo_allWeather, ticker)
    return x_mvo_allWeather

weight_mvo_aw = MVO_AW(df1,df2,df3,df4)

In [51]:
# Risk Parity Block

def rp_obj(y):    #objective function for Risk parity
    kappa = 5
    return np.matmul(Q,y)-kappa*np.reciprocal(y) #using the gradient of f(y) = 0

def rp_opt(df1,df2,df3,df4):
    total_df, ticker = dataSlicing(df1,df2,df3,df4)
    Q = total_df.cov()
    numAsset = total_df.shape[1]
    
    guess = pd.DataFrame(np.repeat(1/numAsset, numAsset))
    y = scipy.optimize.fsolve(rp_obj,guess)
    
    x_rp=[]
    y_sum = np.sum(y)
    for i in range(len(y)):
        x_rp.append(y[i]/y_sum)
    x_rp = formDf(x_rp, ticker)
    
    return x_rp

weight_rp = rp_opt(df1,df2,df3,df4)

NameError: name 'numAsset' is not defined

In [ ]:
# Naive Risk Parity Block

def naive_rp(df1,df2,df3,df4):
    total_df, ticker = dataSlicing(df1,df2,df3,df4)
    Q = total_df.cov()
    numAsset = total_df.shape[1]
    
    sigma_sq = np.diagonal(Q)# variance for all assets (sigma^2 )
    #1/sigma^2
    inv_sigma_sq = pd.DataFrame(sigma_sq).rdiv(1).T 

    # 1/sigma
    inv_sigma = inv_sigma_sq.apply(np.sqrt)
    inv_sigma= np.array(inv_sigma.iloc[0,:].tolist())
    
    x_naive_rp = []
    denom = np.sum(inv_sigma) #sum of 1/sigma at the denominator

    for i in range(numAsset):
        x_naive_rp.append((inv_sigma[i]/denom))

    x_naive_rp = formDf(x_naive_rp, ticker)
    return x_naive_rp

weight_naive_rp = naive_rp(df1,df2,df3,df4)